In [1]:
import dhlab as dh
import dhlab.api.dhlab_api as api
import pandas as pd
from dhlab.nbtokenizer import tokenize
import jsonlines
import random
dh.css()

# Lemmatize a set

a = dh.totals(100000)
lemma = dh.WordLemma(list(a.index)).lemmas
reduced_set = pd.DataFrame(lemma.merge(a.reset_index(), left_on=0, right_on='index').groupby(2)['freq'].max())
lower_set = pd.DataFrame(reduced_set.groupby(lambda x: x.lower())['freq'].sum())
lower_set.sort_values(by='freq', ascending=False)

# Generate sentences

In [2]:
def make_dewey_corpus(from_year = 1980, to_year = 2000, limit_pr_dewey=3000):
    corpus = dh.Corpus(doctype='digibok', lang='nob NOT (mul OR eng OR ger OR dan OR swe)', ddk='0*', from_year=from_year, to_year=to_year, limit = limit_pr_dewey)
    for i in range(1,10):
        dewey = f"{str(i)}*"
        #print(dewey)
        corpus.add(dh.Corpus(doctype='digibok', lang='nob NOT (mul OR eng OR ger OR dan OR swe)', order_by='first', ddk=dewey, from_year=from_year, to_year = to_year, limit = limit_pr_dewey))
    print(f"Totalt antall URN'er {corpus.size}")
    return corpus

In [ ]:
corpus = dh.Corpus(doctype='digibok', lang='nob NOT (mul OR eng OR ger OR dan OR swe)', ddk='3*', from_year=1980, limit = 100000)
corpus.size

In [4]:
corpus_med = dh.Corpus(doctype='digibok', lang='nob NOT (mul OR eng OR ger OR dan OR swe)', subject='medisin OR helse', from_year=1980, limit = 100000)
corpus_med.size

3292

In [5]:
def generate_word_sequences_from_corpus(corpus, samples, limit):
    sentences = dh.Concordance(corpus.sample(samples), query = "og OR i OR på OR av OR eller OR har OR ha OR hatt OR er OR var", limit = limit)
    a = sentences.show(style=False, n=limit).concordance.apply(lambda x:x.replace('<b>','').replace('</b>','')).values
    return a

In [78]:
def generate_word_sequences(filename, corpus, samples, limit):
    # sample in four, using period or comma before or after
    # limit pr. corpus
    
    # here params are the choices of before and after and word
    # This will ensure a mix of a selection of sentences 
    # samples are evenly distributed over the parameters
    
    params = [(['.',',','og'], 0, 24), (['.',',','og'], 24, 0)]
    
    if limit < len(params):
        limit = len(params)
    j = 0
    
    sampling = min(samples, len(corpus))
    
    sample_books = [int(x) for x in random.choices(list(corpus.sample(samples).dhlabid.values), k=sampling)]
    print(sample_books[:5])
    for i in range(len(params)):
        words = params[i][0]
        before = params[i][1]
        after = params[i][2]
        a = api.word_concordance(dhlabid = sample_books, words = words, before = before, after = after, limit = int(limit/len(params)))
        with open(filename, "w") as outfile:
            for _, row in a.fillna('').iterrows():
                j += 1
                outfile.write(row.before)
                outfile.write(row.target)
                outfile.write(row.after)
                outfile.write('\n')
    print(f"Wrote {j} lines to {filename}")

In [65]:
tc = make_dewey_corpus(from_year = 1980, to_year = 2020, limit_pr_dewey=40000)

Totalt antall URN'er 143848


In [6]:
#dhids = [ 100399477, 100071037, 100136829, 100092673]
#a = api.word_concordance(dhlabid = [100092673], words = ['.',','], before = 0, after = 10, limit = 50)

In [7]:
#generate_word_sequences("dewey_all_tst.txt", tc.frame, 1200, 90)

In [8]:
def convert(sentence, lower=False):
    """Tokenize i a string, turn all tokens to lower case and 
    remove punctuations and other stuff not normally part of a word"""
    lst = tokenize(sentence)
    lowered = [x for x in lst if x.isalpha()]
    return lowered

## Generate json lines

In [152]:
dewey = "4"
corpus = dh.Corpus(doctype='digibok', lang='nob NOT (mul OR eng OR ger OR dan OR swe)', ddk=f'{dewey}*', from_year=1980, limit = 100000)
corpus.size

2194

In [9]:
subject = "medisin OR helse"
corpus = dh.Corpus(doctype='digibok', lang='nob NOT (mul OR eng OR ger OR dan OR swe)', subject=subject, from_year=1980, limit = 100000)
corpus.size

3292

In [10]:
%%time
for i in range(400):
    if i%100 == 0:
        print(i)
    a = generate_word_sequences_from_corpus(corpus, 100, 3000)
    with open(f"extract_subject_{'-'.join(subject.split())}.txt",mode='a') as wr:
        for sent in map(convert, a):
            wr.write(' '.join(sent))
            wr.write('\n')
    

0
100
200
300
CPU times: user 1min 26s, sys: 855 ms, total: 1min 27s
Wall time: 16min 34s
